<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-файл-с-данными-и-изучите-общую-информацию." data-toc-modified-id="Откройте-файл-с-данными-и-изучите-общую-информацию.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте файл с данными и изучите общую информацию.</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Посчитайте-и-добавьте-в-таблицу-новые-столбцы" data-toc-modified-id="Посчитайте-и-добавьте-в-таблицу-новые-столбцы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Посчитайте и добавьте в таблицу новые столбцы</a></span></li><li><span><a href="#Проведите-исследовательский-анализ-данных" data-toc-modified-id="Проведите-исследовательский-анализ-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проведите исследовательский анализ данных</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование объявлений о продаже квартир

В нашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Наша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

### Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd # импортируем библиотеку pandas
import matplotlib.pyplot as plt # импортируем библиотеку matplotlib
import numpy as np # импортируем библиотеку numpy

In [2]:
data = pd.read_csv('/datasets/real_estate_data.csv') # загружаем данные из файла в датафрейм
data.head() # выводим его первые строки

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/real_estate_data.csv'

Видим, что исходные данные не разбиты по колонкам

In [ ]:
data = pd.read_csv('/datasets/real_estate_data.csv', sep="\t") # разделяем колонки
data.head() # снова выводим первые строки таблицы, чтобы увидеть результат

In [ ]:
data.info() # выводим общую информацию о полученном датафрейме

Датафрейм содержит 23 699 записей (22 столбца), при этом в ряде столбцов (ceiling_height, floors_total, living_area, is_apartment, kitchen_area, balcony, locality_name, airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition) значения пропущены.
При этом следует отметить, что формат значений в отдельных столбцах не соответствует требуемому: например, first_day_exposition (дата публикации) - object вместо datetime64, floors_total (всего этажей в доме) - float64 вместо int64, is_apartment (апартаменты) - object вместо bool, balcony (число балконов) - float64 вместо int64, parks_around3000 (число парков в радиусе 3 км) - float64 вместо int64, ponds_around3000 (число водоёмов в радиусе 3 км) - float64 вместо int64, days_exposition (сколько дней было размещено объявление) - float64 вместо int64.
Кроме того, некоторые столбцы лучше переименовать (cityCenters_nearest, parks_around3000, ponds_around3000).

In [ ]:
data.hist(figsize=(15, 20)) # строим общую гистограмму для всех числовых столбцов таблицы

На гистограммах уже видим возможные аномалии (например, столбцы last_price, balcony).

### Предобработка данных

**2.1 Поиск и изучение пропущенных значений в столбцах**

In [ ]:
data.isna().sum() # определяем, в каких столбцах есть пропуски, а также их количество

In [ ]:
data['balcony'] = data['balcony'].fillna(0) # если продавец не указал число балконов, то, скорее всего, в его квартире их нет. Такие пропуски заменяем на 0.
data['balcony'] = data['balcony'].astype(int) # меняем тип данных в столбце на int64 (количество балконов может быть только целым числом).
data['is_apartment'] = data['is_apartment'].astype(bool) # меняем тип данных в столбце на bool (недвижимость либо является апартаментами, либо нет).
data['is_apartment'] = data['is_apartment'].fillna(False) # если продавец не указал, является ли недвижимость апартаментами, то, скорее всего, не является. Такие пропуски заменяем на False.
data['parks_around3000'] = data['parks_around3000'].fillna(0) # если продавец не указал число парков поблизости (либо количество не "подтянулось" автоматически), то, скорее всего, их нет. Такие пропуски заменяем на 0.
data['ponds_around3000'] = data['ponds_around3000'].fillna(0) # если продавец не указал число водоемов поблизости (либо количество не "подтянулось" автоматически), то, скорее всего, их нет. Такие пропуски заменяем на 0.
data['parks_around3000'] = data['parks_around3000'].astype(int) # меняем тип данных в столбце на int64 (количество парков может быть только целым числом).
data['ponds_around3000'] = data['ponds_around3000'].astype(int) # меняем тип данных в столбце на int64 (количество водоемов может быть только целым числом).

Отметим, что количество пропусков в столбце parks_around3000 и столбце ponds_around3000 идентично - скорее всего, пропуски в этих столбцах присутствуют по одним и тем же объектам (соответствующие объекты находятся вдали от парков и водоемов).

In [ ]:
data = data.dropna(subset=['locality_name']) # удалим 49 квартир с пропущенными сведения о населенном пункте (данная информация слишком существенна для анализа)

In [ ]:
data = data.dropna(subset=['floors_total']) # удалим 86 квартир с пропущенными сведения об этажности дома (данная информация слишком существенна для анализа)

In [ ]:
no_kitchen = data[data['kitchen_area'].isnull()]
no_kitchen['is_apartment'].value_counts() 

Видим, что большинство квартир "без кухни" является апартаментами.

In [ ]:
no_kitchen['open_plan'].value_counts()

Видим, что квартир со свободной планировкой с пропущенной площадью кухни немного.

In [ ]:
no_living = data[data['living_area'].isnull()]
no_living['is_apartment'].value_counts()

Видим, что большинство квартир "без жилой площади" является апартаментами.

In [ ]:
no_living['open_plan'].value_counts()

Видим, что квартир со свободной планировкой с пропущенной жилой площадью очень мало.

In [ ]:
avg_total = data.loc[(~data['living_area'].isna()), 'total_area'].mean()
avg_living = data.loc[(~data['living_area'].isna()), 'living_area'].mean()
ratio_1 = avg_total / avg_living
data['living_area'] = data['living_area'].fillna(data['total_area'] / ratio_1) # заменим пропуски в столбце living_area на приблизительное вероятное значение, рассчитанное с помощью коэффициента (получен на основе выборки квартир без пропусков в данном столбце с использованием средних значений площади)

In [ ]:
avg_total_1 = data.loc[(~data['kitchen_area'].isna()), 'total_area'].mean()
avg_kitchen = data.loc[(~data['kitchen_area'].isna()), 'kitchen_area'].mean()
ratio_2 = avg_total_1 / avg_kitchen
data['kitchen_area'] = data['kitchen_area'].fillna(data['total_area'] / ratio_2) # заменим пропуски в столбце kitchen_area на приблизительное вероятное значение, рассчитанное с помощью коэффициента (получен на основе выборки квартир без пропусков в данном столбце с использованием средних значений площади)

Количество пропусков в столбце ceiling_height слишком велико (почти 40% выборки) - целесообразнее оставить их как есть, чтобы не исказить статистики выборки и результаты анализа.

**2.2 Причины пропусков в данных**

По всей видимости, по ряду объектов картографические данные (расстояние до центра города, аэропорта и ближайшего водоема/парка) отсутствуют вследствие технической ошибки (данные сведения не вносятся пользователями). Данные о расстоянии до центра города могут отсутствовать в отношении объектов, находящихся в небольших населенных пунктах. Можно предположить, что название населённого пункта также не "подтянулось" в базу из общего справочника из-за технического сбоя.
Ряд данных, которые должны были быть внесены пользователями, также отсутствует по отдельным объектам: ceiling_height (высота потолков), floors_total (всего этажей в доме), living_area (жилая площадь в квадратных метрах), kitchen_area (площадь кухни).
Указанное, возможно, обусловлено человеческим фактором (пользователи забыли заполнить соответствующие ячейки) либо тем, что параметры не были известны пользователям либо они посчитали эти сведения несущественными.
Стоит отметить, что в отдельных случаях данные о kitchen_area могли отсутствовать из-за отсутствия отдельной кухни в квартире (апартаменты, свободная планировка).
Кол-во дней, в течение которых было размещено объявление, отсуствует по причине того, что на момент среза объявление еще не было снято.

**2.3 Меняем типы данных в иных столбцах**

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%d %H:%M') # меняет формат данных в столбце на datetime64 (так как это дата) и избавляемся от нулевых значений часа/минуты/секунды.  
data = data.rename(columns={'cityCenters_nearest': 'city_centers_nearest', 'parks_around3000': 'parks_around_3000', 'ponds_around3000': 'ponds_around_3000'}) # приводим в порядок названия некоторых столбцов

Формат данных в столбцах floors_total и days_exposition поменять на int64 невозможно, так как в них есть пропуски, которые в соответствии с инструкцией по выполнению проекта нужно оставить (логичную замену предложить невозможно).

**2.4 Устраняем неявные дубликаты в столбце с названиями**

In [ ]:
print(data.duplicated().sum()) # проверим, нет ли явных дубликатов

In [ ]:
print(data['locality_name'].unique()) # изучаем уникальные значения в столбце с названиями населенных пунктов
data['locality_name'] = data['locality_name'].str.replace('ё', 'е')
data['locality_name'] = data['locality_name'].str.replace('городской поселок', 'поселок городского типа')
data['locality_name'] = data['locality_name'].str.replace('поселок при железнодорожной станции', 'поселок станции')
data['locality_name'] = data['locality_name'].str.replace('садоводческое некоммерческое товарищество', 'садовое товарищество')
# устраняем неявные дубликаты

**2.5 Устраняем редкие и выбивающиеся значения (аномалии) во всех столбцах**

In [ ]:
print(data['total_images'].describe()) # получаем характеристики, дающие представление о выборке
data['total_images'].hist() # строим гистограмму

В отношении количества фотографий ничего необычного не выявлено (есть объявления без фотографий).

In [ ]:
print(data['total_area'].describe()) # получаем характеристики, дающие представление о выборке
data['total_area'].hist() # строим гистограмму

Настораживают объекты общей площадью 12 кв м и 900 кв м.

In [ ]:
display(data.query('total_area == 12')) # получаем строку с объектом площадью 12 кв м.

Учитывая, что это апартаменты - похоже на правду.

In [ ]:
display(data.query('total_area == 900'))

Учитывая сочетание остальных параметров - похоже на правду.

In [ ]:
print(data['last_price'].describe()) # получаем характеристики 
data['last_price'].head() # выводим несколько значений

Функция выводит данные в странном формате (числа слишком длинные). Видим, что в таблице данные отображаются нормально.

In [ ]:
data['last_price'].max()

Получили самую высокую цену в выборке - 763 млн руб.

In [ ]:
display(data.query('last_price > 100000000 and total_area < 120'))

Видим, что дорогих квартир площадью менее 120 кв. м в базе нет.

In [ ]:
data['last_price'].min() 

Получили самую низкую цену в выборке - она подозрительно невысокая.

In [ ]:
display(data.query('last_price < 500000')) # отбираем квартиры стоимостью до 500 тыс. руб.

Стоимость двухкомнатной квартиры в Санкт-Петербурге площадью 109 кв. м в размере 12 190 руб. очевидно ошибочная.
Вероятно, она стоит 12 190 000 руб. Остальные объекты гораздо меньше по площади, стоят все-таки дороже и находятся в небольших населенных пунктах Ленинградской области.

In [ ]:
rows_12 = (data['last_price'] == 12190.0)
data.loc[rows_12, "last_price"] = 12190000.0 # меняем стоимость квартиры

In [ ]:
data['last_price'].hist(bins=100, range=(500000, 50000000)) # строим гистограмму

In [ ]:
print(data['rooms'].describe()) # получаем характеристики
data['rooms'].hist() # строим гистограмму

0 комнат и 19 комнат вызывают сомнения.

In [ ]:
display(data.query('rooms == 0'))

Квартир с 0 комнат у нас в выборке довольно много (194 или почти 1%). Надо разобраться.

In [ ]:
rooms_0 = data.query('rooms == 0')
rooms_0.query('total_area <= 45')

Общая площадь 189 из 194 квартир без комнат не превышает 45 кв м - возможно заменить значение 0 на 1 (скорее всего это однушки-студии со свободной планировкой).

In [ ]:
rows = (data['rooms'] == 0) & (data['total_area'] <= 45)
data.loc[rows, "rooms"] = 1 # делаем "однокомнатными" квартиры площадью не более 45 кв м

In [ ]:
rooms_0.query('total_area > 45')

5 апартаментов площадью от 58,4 кв м до 371 кв м вряд ли являются однокомнатными, логичную замену нулю предположить затруднительно.

In [ ]:
data = data[(data['rooms'] != 0) | (data['total_area'] < 45)] # убираем строки с 'большими' квартирами без комнат (5 квартир - несущественное для выборки количество)

In [ ]:
display(data.query('rooms > 10'))

Видим, что площадь квартир с количеством комнат от 10 не меньше 183,7 кв. м. Цена тоже соответствующая.

In [ ]:
print(data['ceiling_height'].describe()) # выводим характеристики
data['ceiling_height'].hist(bins=10, range=(1,30)) # строим гистограмму

Видим, что в базе есть квартиры с гигантскими потолками до 100 метров - похоже на ошибку.
Также видим, что в базе есть квартиры с аномально низкими потолками (1 м как минимум).

In [ ]:
display(data.query('ceiling_height > 10'))

Квартир с аномально большими потолками в базе 25.
Потолки высотой 22,6 м и выше возможно заменить на значения, разделенные на 10 (скорее всего, высота этих потолков 2,26 м и т.д.). Однако, чем заменить значения 10,3, 14, 20 и 100 м - неясно. Целесообразно их удалить.

In [ ]:
data = data[(data['ceiling_height'] != 10.3) & (data['ceiling_height'] != 14) & (data['ceiling_height'] != 20) & (data['ceiling_height'] != 100)] # убираем строки с аномально высокими потолками (4 квартиры - несущественное для выборки количество)

In [ ]:
rows_1 = (data['ceiling_height'] > 10)
data.loc[rows_1, "ceiling_height"] = data.loc[rows_1, "ceiling_height"] / 10 # делим значения высоты потолков больше 10 м на 10, чтобы привести их к реальным

In [ ]:
display(data.query('ceiling_height < 2'))

В базе обнаружены 3 квартиры с аномально низким потолком (менее 2 м). Целесообразно их удалить из базы.

In [ ]:
data = data[(data['ceiling_height'] != 1) & (data['ceiling_height'] != 1.20) & (data['ceiling_height'] != 1.75)]  # удаляем 3 объекта с высотой потолка менее 2 м

In [ ]:
print(data['floors_total'].describe()) # получаем характеристики столбца
data['floors_total'].hist() # строим гистограмму

В базе данных присутствуют здания с количеством этажей от 1 до 60 - похоже на правду.

In [ ]:
print(data['living_area'].describe()) # получаем характеристики столбца
data['living_area'].hist() # строим гистограмму

Вызывает вопросы минимальное значение жилой площади (2 кв м), максимальное тоже можно проверить (409 кв м).

In [ ]:
display(data.query('living_area < 10 and total_area > 40'))

Выборка квартир с жилой площадью менее 10 кв. м и общей площадью более 40 кв. м вызывает вопросы - эти данные не бьются с показателями общей площади квартиры в совокупности с площадью кухни и количеством комнат: например, в выборке есть трехкомнатная квартира общей плащадью 139 кв. м с кухней площадью 16 кв. м и жилой площадью 3 кв. м. По отдельным объектам значения площади кухни и жилой площади возможно перепутаны местами (например, мы видим двухкомнатную квартиру с кухней площадью 41 кв. м и жилой площадью 8,4 кв. м), в ряде случаев возможно указано 2.0 вместо 20, например. В целом природа данной аномалии непонятна.

In [ ]:
data = data[~(data['living_area'] < 10) | ~(data['total_area'] > 40)] # удаляем из базы 10 квартир с жилой площадью менее 10 кв. м и общей площадью более 40 кв. м

In [ ]:
display(data.query('living_area > 300'))

Учитывая остальные параметры, похоже на правду.

In [ ]:
print(data['floor'].describe()) # получаем характеристики столбца
data['floor'].hist() # строим гистограмму

В части этажа, на котором находится квартира, аномалий не выявлено (минимальный - 1-й, максимальный - 33-й).

In [ ]:
print(data['kitchen_area'].describe()) 
data['kitchen_area'].hist()

Надо проверить слишком малые и большие значения площади кухни.

In [ ]:
print(data.query('kitchen_area > 50 and total_area < 100'))

Маленьких квартир с аномально большой кухней не обнаружено.

In [ ]:
display(data.query('kitchen_area < 3'))

В случае, если данные квартиры являются апартаментами, согласиться можно. Однако одна квартира (не апартаменты) площадью 66,4 кв. м с кухней площадью 2,89 кв м выглядит аномально.

In [ ]:
data = data[~(data['kitchen_area'] == 2.89) | ~(data['total_area'] == 66.40)] # убираем вышеуказанную квартиру

In [ ]:
print(data['balcony'].describe()) # получаем характеристики столбца
display(data.query('balcony > 3')) # выводим квартиры с количеством балконов больше 3

Видим подозрительно большое количество квартир с большим количеством балконов (больше 3, доходит до 5).
Особенно смущает несоответствие количества балконов количеству комнат. Возможно предположить, что пользователи в ряде случаев вносили данные о площади балкона, а не о количестве балконов.

In [ ]:
display(data.query('balcony > 3 and rooms < 3'))

В выборке 184 квартиры, у которых меньше 3 комнат и больше 3 балконов!

In [ ]:
display(data.query('balcony > 4 and rooms < 2'))

В выборке есть даже 24 однушки с 5 балконами!

С балконами надо что-то делать. Квартир с аномальным количеством балконов слишком много, чтобы удалить их из выборки.
Целесообразно выбивающиеся значения привести к адекватным - например, сделать так, чтобы количество балконов не превышало количество комнат больше чем на 1 (+ гипотетический балкон для кухни)

In [ ]:
rows_b = (data['rooms'] == 1) & (data['balcony'] > 2)
data.loc[rows_b, "balcony"] = 2
rows_b2 = (data['rooms'] == 2) & (data['balcony'] > 3)
data.loc[rows_b2, "balcony"] = 3
rows_b3 = (data['rooms'] == 3) & (data['balcony'] > 4)
data.loc[rows_b3, "balcony"] = 4 # меняем соответствующие данные

In [ ]:
print(data['airports_nearest'].describe()) # получаем характеристики столбца

Вызывает вопросы квартира, находящася фактически внутри аэропорта.

In [ ]:
display(data.query('airports_nearest < 6000'))

Менее 6 км до ближайшего аэропорта только от 1 квартиры - а именно, 0 м.
Показатель картографический - видимо, ноль появился из-за технической ошибки.

In [ ]:
data = data[~(data['airports_nearest'] == 0)] # убираем вышеуказанную квартиру

In [ ]:
print(data['city_centers_nearest'].describe()) # получаем характеристики столбца

Данные не вызывают подозрений.

In [ ]:
print(data['ponds_around_3000'].describe()) # получаем характеристики столбца

Данные не вызывают подозрений.

In [ ]:
print(data['ponds_nearest'].describe()) # получаем характеристики столбца

Данные не вызывают подозрений (хотя водоем в 13 м от дома - это довольно близко).

In [ ]:
print(data['parks_around_3000'].describe()) # получаем характеристики столбца

Данные не вызывают подозрений.

In [ ]:
print(data['parks_nearest'].describe()) # получаем характеристики столбца

Данные не вызывают подозрений (дом в 1 метре от парка находиться может).

In [ ]:
print(data['days_exposition'].describe()) # получаем характеристики столбца

Данные не вызывают подозрений.

In [ ]:
data = data.reset_index(drop=True) # обновляем индексацию после удаления строчек
data.info()

Видим, что количество записей уменьшилось до 23 541 (в целом несущественно). Однако в ряде столбцов сохранились пропуски, которые заменить невозможно.

### Посчитайте и добавьте в таблицу новые столбцы

**3.1 Добавляем в таблицу данные о цене одного квадратного метра**

In [ ]:
m_p = data['last_price'] / data['total_area'] # чтобы найти цену одного квадратного метра, нужно разделить ощую стоимость на общую площадь
m_p = m_p.round() # округляем
data.insert(2, 'meter_price', m_p, allow_duplicates = False) # вставляем столбец в нужное место
data.head() # проверяем, что получилось

**3.2 Добавляем в таблицу данные о дне публикации объявления**

In [ ]:
weekday = data['first_day_exposition'].dt.weekday # определяем день
data.insert(5, 'weekday', weekday, allow_duplicates = False) # вставляем столбец в нужное место
data.head() # проверяем, что получилось

**3.3 Добавляем в таблицу данные о месяце публикации объявления**

In [ ]:
month = pd.DatetimeIndex(data['first_day_exposition']).month # вычленяем месяц
data.insert(6, 'month', month, allow_duplicates = False) # вставляем столбец в нужное место
data.head() # проверяем, что получилось

**3.4 Добавляем в таблицу данные о годе публикации объявления**

In [ ]:
year = pd.DatetimeIndex(data['first_day_exposition']).year # вычленяем год
data.insert(7, 'year', year, allow_duplicates = False) # вставляем столбец в нужное место
data.head() # проверяем, что получилось

**3.5 Добавляем в таблицу данные о типе этажа квартиры**

In [ ]:
def type_floor(floor):   # присваиваем каждой квартире тип этажа с помощью функции
    if floor['floor'] == 1:
        return 'первый'
    if floor['floor'] == floor['floors_total']:
        return 'последний'
    return 'другой'
type_f = data.apply(type_floor, axis=1)
data.insert(13, 'type_floor', type_f, allow_duplicates = False) # вставляем столбец в нужное место
data.head() # проверяем, что получилось

**3.6 Добавляем в таблицу данные о расстоянии до центра города в километрах**

In [ ]:
centre_km = data['city_centers_nearest'] / 1000 # чтобы получить сведения о расстоянии в километрах, делим имеющееся расстояние в метрах на 1000
centre_km = centre_km.round() # округляем
data.insert(22, 'centre_km', centre_km, allow_duplicates = False) # вставляем столбец в нужное место

### Проведите исследовательский анализ данных

**4.1 Изучение параметров выборки, в том числе с помощью гистограмм**

*4.1.1 Общая площадь*

In [ ]:
data['total_area'].describe() # выводим характеристики столбца.

Cреднее арифметическое - 60,3 кв. м, стандартное отклонение - 35,6 кв., минимальное значение - 12 кв. м, максимальное - 900 кв м, медиана - 52 кв м.

In [ ]:
data['total_area'].hist(bins=100, range=(0, 150)) # строим гистограмму

Видим, что основная масса значений лежит в диапазоне 30 - 80 кв. м, что выглядит реалистично. График напоминает нормальное распределение.

*4.1.2 Жилая площадь*

In [ ]:
data['living_area'].describe() # выводим характеристики столбца.

Среднее арифметическое - 34,6 кв. м, стандартное отклонение - 22,4 кв., минимальное значение - 2 кв. м, максимальное - 409 кв м, медиана - 30 кв м.
В целом цифры коррелируют со значениями общей площади.

In [ ]:
data['living_area'].hist(bins=100, range=(0, 60)) # строим гистограмму

Видим, что основная масса значений лежит в диапазоне 15 - 50 кв. м, что выглядит реалистично.
На графике можно отметить "провал" в районе площади 20-27 кв м - таких квартир очень мало в выборке.

*4.1.3 Площадь кухни*

In [ ]:
data['kitchen_area'].describe() # выводим характеристики столбца.

Среднее арифметическое - 10,5 кв. м, стандартное отклонение - 6 кв., минимальное значение - 1,3 кв. м, максимальное - 112 кв м, медиана - 9 кв м.
В целом цифры коррелируют со значениями общей площади и жилой площади.

In [ ]:
data['kitchen_area'].hist(bins=100, range=(0, 20)) # строим гистограмму

Видим, что основная масса значений лежит в диапазоне 6 - 15 кв. м, что выглядит реалистично.
Определенные значения площади кухни существенно более распространены (6, 8, 11 кв. м)

*4.1.4 Цена объекта*

In [ ]:
data['last_price'].describe() # выводим характеристики столбца

Самый дорогой объект в выборке стоит 763 млн руб., самый дешевый - 430 тыс. руб., медианное значение цены - чуть больше 4,6 млн руб., что ожидаемо, учитывая локацию. Среднее значение (6,5 млн руб.) больше медианного за счет "дорогих" выбросов.

In [ ]:
data['last_price'].hist(bins=100, range=(1500000, 50000000), figsize=(10,10)) # строим гистограмму

Гистограмма нечитабельна из-за длины чисел.

In [ ]:
data['last_price_mln'] = data['last_price'] / 1000000 # для удобства создадим столбец с ценой в миллионах рублей

In [ ]:
data['last_price_mln'] = data['last_price_mln'].round(1) # округлим

In [ ]:
data['last_price_mln'].hist(bins=50, range=(1, 50), figsize=(10,10)) # строим гистограмму по ценам в миллионах рублей

Видим, что стоимость большинства квартир составляет около 5 млн руб.

*4.1.5 Количество комнат*

In [ ]:
data['rooms'].describe() # выводим характеристики столбца.

Cреднее арифметическое - 2,1, стандартное отклонение - 1,1, минимальное значение - 1, максимальное - 19, медиана - 2, третий квартиль - 3.
В целом цифры ожидаемые (квартиры с количеством комнат больше 10 были рассмотрены выше)

In [ ]:
data['rooms'].hist(bins=20, range=(1, 5)) # строим гистограмму 

Видим, что основная масса квартир - однокомнатные и двухкомнатные, трехкомнатных квартир немного меньше, что выглядит реалистично.

*4.1.6 Высота потолков*

In [ ]:
data['ceiling_height'].describe() # выводим характеристики столбца.

Среднее арифметическое - 2,7, стандартное отклонение - 0,3, минимальное значение - 2, максимальное - 8,3, медиана - 2,65, третий квартиль - 2,8. В целом цифры ожидаемые.

In [ ]:
data['ceiling_height'].hist(bins=20, range=(2, 3)) # строим гистограмму

Видим, что основная масса квартир имеет высоту потолка 2,5 м и чуть больше, что выглядит реалистично. Вместе с тем, необходимо помнить, что в базе существенное количество строк (более 9 тыс.) имеют пропуски в столбце с высотой потолка.

*4.1.7 Этаж квартиры*

In [ ]:
data['floor'].describe() #  выводим характеристики столбца.

Среднее арифметическое - 5,9, стандартное отклонение - 4,9, минимальное значение - 1, максимальное - 33, медиана - 4, третий квартиль - 8. В целом цифры ожидаемые.

In [ ]:
data['floor'].hist(bins=30, range=(1, 20)) # строим гистограмму

Видим, что квартиры в основном расположены на 1-5 этажах.

*4.1.8 Тип этажа квартиры*

In [ ]:
data['type_floor'].value_counts()

Видим, что 73,5% квартир располагаются не на первом и не на последнем этаже. Квартир на последнем этаже чуть больше, чем на первом.

In [ ]:
data['type_floor'].hist(bins=3) 

Гистограмма отражает вышеуказанные цифры.

*4.1.9 Общее количество этажей в доме*

In [ ]:
data['floors_total'].describe() # выводим характеристики столбца.

Среднее арифметическое - 10,7, стандартное отклонение - 6,6, минимальное значение - 1, максимальное - 60, медиана - 9, третий квартиль - 16. В целом цифры ожидаемые.

In [ ]:
data['floors_total'].hist(bins=8, range=(1, 15)) 

График показывает, что пяти- и девятиэтажек больше всего (ожидаемо).

*4.1.10 Расстояние до центра города в метрах*

In [ ]:
data['city_centers_nearest'].describe() # выводим характеристики столбца.

Среднее арифметическое - 14 194, стандартное отклонение - 8 614, минимальное значение - 181, максимальное - 65 968, медиана - 13 099, третий квартиль - 16 285. В основном квартиры расположены не в самом центре.

In [ ]:
data['city_centers_nearest'].hist(bins=30, range=(100, 30000)) 

На гистограмме видим, что квартир в 5 км от центра больше, чем, например, 7-8 км. В основном квартиры расположены в 13-15 км от центра.

*4.1.11 Расстояние до ближайшего аэропорта*

In [ ]:
data['airports_nearest'].describe() # выводим характеристики столбца

Среднее арифметическое - 28 799, стандартное отклонение - 12 633, минимальное значение - 6 450, максимальное - 84 869, медиана - 26 757, третий квартиль - 37 282.
В основном квартиры расположены относительно далеко от аэропорта - в среднем 27-28 км (гораздо дальше, чем от центра города - аэропорт находится за городом).

In [ ]:
data['airports_nearest'].hist(bins=30, range=(10000, 70000)) 

На гистограмме видим, что квартиры в основном расположены примерно в 15-35 км от ближайшего аэропорта.

*4.1.12 Расстояние до ближайшего парка*

In [ ]:
data['parks_nearest'].describe() # выводим характеристики столбца.

Среднее арифметическое - 490, стандартное отклонение - 341, минимальное значение - 1, максимальное - 3 190, медиана - 455, третий квартиль - 612.

In [ ]:
data['parks_nearest'].hist(bins=30, range=(100, 1000))

Судя по гистограмме, квартиры расположены гораздо ближе к паркам (400-600 м), чем к аэропорту/центру города, что ожидаемо.

*4.1.13 День и месяц публикации объявления*

In [ ]:
data['weekday'].describe() # выводим характеристики столбца

In [ ]:
data['weekday'].value_counts() 

Для удобства подсчитаем количество уникальных значений. Четверг - самый популярный день.

In [ ]:
data['weekday'].hist(bins=8, range=(0, 7)) 

На графике видим, что объявления публиковались практически равномерно в будние дни, в выходные - гораздо реже, что ожидаемо и логично.

In [ ]:
data['month'].value_counts() # для удобства подсчитаем количество уникальных значений месяца публикации объявления

Самый популярный месяц - февраль. В декабре, январе и мае меньше всего публикаций (в этимх месяцах много праздников, декабрь - предновогодний месяц). Летом публикаций также меньше (сезон отпусков).

In [ ]:
data['month'].hist(bins=30, range=(1, 12)) 

Гистограмма наглядно отражает вышеописанные особенности.

**Согласно результатам анализа, портрет среднестатистической квартиры таков:**
- однушка или двушка на 1-5 этаже пяти- или девятиэтажки (квартир на первом и последнем этажах меньше, чем на промежуточных);
- общая площадь около 45 кв. м;
- жилая площадь около 20 кв. м;
- площадь кухни около 6-10 кв. м;
- цена - около 4,5 млн руб.;
- потолок 2,5 м;
- квартира расположена относительно недалеко от парка (пол-километра), однако не в центре города и в отдалении от аэропорта.
Объявления в основном публикуются в будние дни, скорее не в летние и не в праздничные месяцы.

**4.2 Изучаем, как быстро продавались квартиры**

In [ ]:
data['days_exposition'].hist(bins=70, range=(10, 700), figsize=(20,10)) # строим гистограмму.

Судя по графику, большое количество квартир продавалось через 40 и 60 дней. При этом основная масса - в срок, не превышающий 200 дней.

In [ ]:
data['days_exposition'].mean().round() # считаем округленное среднее

In [ ]:
data['days_exposition'].median() # считаем медиану

In [ ]:
data['days_exposition'].describe() # еще можно таким способом

Исходя из вышеотраженных данных, продажа квартиры занимает в среднем около 6 месяцев (в большинстве случаев - 1-3,5 месяца). Быстрыми продажами можно считать сделки в пределах месяца. Необычно долгие продажи - после 8-9 месяцев.

**4.3  Определяем факторы, которые больше всего влияют на общую (полную) стоимость объекта**

*4.3.1 Корреляция цены и общей площади*

In [ ]:
print(data['last_price_mln'].corr(data['total_area']))

Корреляция (коэффициент Пирсона) 0.65 говорит о наличии связи, но не слишком сильной (используем цены в миллионах для удобства здесь и далее).

In [ ]:
data.plot(x='total_area', y='last_price_mln', style='o', kind='scatter', xlim=(20, 400), ylim=(0, 50), grid=True, figsize=(8, 10))

На графике видим, что в выборке есть дорогие (дороже 20 млн руб.) квартиры как большой площади (более 200 кв м), так и относительно небольшие (100-150 кв м). Видим, что квартира стоимостью 40 млн руб, например, может быть площадью как 70 кв м, так и 300 кв м.

*4.3.2 Корреляция цены и жилой площади* 

In [ ]:
print(data['last_price_mln'].corr(data['living_area']))

Корреляция (коэффициент Пирсона) - 0,585 меньше, чем корреляция цены с общей площадью. Связь сохраняется, но она чуть менее прямая.

In [ ]:
data.plot(x='living_area', y='last_price_mln', style='o', kind='scatter', xlim=(15, 200), ylim=(0, 50), grid=True, figsize=(8, 11))

График напоминает график соотношения общей площади и цены и выглядит логично на фоне более низкого коэффициента корреляции.
В выборке небольших дорогих квартир не меньше, чем больших.

*4.3.3 Корреляция цены и площади кухни*

In [ ]:
print(data['last_price_mln'].corr(data['kitchen_area']))

Корреляция (коэффициент Пирсона) еще меньше - 0.51. Площадь кухни является менее существенным фактором, нежели общая площадь и жилая площадь.

In [ ]:
data.plot(x='kitchen_area', y='last_price_mln', style='o', kind='scatter', xlim=(5, 50), ylim=(0, 50), grid=True, figsize=(8, 10))

График напоминает графики соотношения общей/жилой площади и цены и выглядит логично на фоне еще более низкого коэффициента корреляции.
В выборке дорогих квартир с небольшой кухней не меньше, чем с большой.

*4.3.4 Корреляция цены и количества комнат*

In [ ]:
print(data['rooms'].corr(data['last_price_mln']))

Корреляция (коэффициент Пирсона) слабая -  само по себе большое количество комнат не сильно увеличивает стоимость.

In [ ]:
data_pivot_r = data.pivot_table(index=['rooms'], values='last_price_mln', aggfunc=['count', 'median']) # создадим сводную таблицу
data_pivot_r

Видим, что 1-3 комнатные квартиры составляют основную часть выборки - при этом медианная цена ожидаемо растет с увеличением количества квартир.

In [ ]:
data_pivot_r.plot(kind='bar', ylim=(0, 5000), figsize=(10,10)) # строим диаграмму по данным сводной таблицы

Видим, что 1-3 комнатные квартиры составляют подавляющее большинство объектов. Медианная цена 12-комнатной квартиры выделяется за счет выброса (такая квартира в выборке всего одна). 

*4.3.5 Корреляция цены и этажа, на котором расположена квартира (первый, последний, другой)*

In [ ]:
data_pivot_f = data.pivot_table(index=['type_floor'], values='last_price_mln', aggfunc=['median']) # создаем сводную таблицу
data_pivot_f

In [ ]:
data_pivot_f.plot()

Медианная цена квартир, находящихся не на первом и не напоследнем этаже, выше на 10-20%. Квартиры на первом этаже в среднем дешевле квартир на последнем этаже, что ожидаемо.

*4.3.6 Корреляция цены и даты размещения (день недели, месяц, год)*

In [ ]:
data_pivot_w = data.pivot_table(index=['weekday'], values='last_price_mln', aggfunc=['median']) # создаем сводную таблицу
data_pivot_w

In [ ]:
data_pivot_w_1 = data.pivot_table(index=['weekday'], values='last_price_mln', aggfunc=['count'])
data_pivot_w_1

In [ ]:
data_pivot_w.plot()

In [ ]:
print(data['last_price_mln'].corr(data['weekday'])) 

Корреляции почти нет. Видим, что цена почти не зависит от дня недели, в который было размещено объявление (это логично).

In [ ]:
data_pivot_m = data.pivot_table(index=['month'], values='last_price_mln', aggfunc=['median'])
data_pivot_m

In [ ]:
print(data['last_price_mln'].corr(data['month'])) 

Корреляции почти нет

In [ ]:
data_pivot_m.plot()

По итогам анализа видим, что корреляции цены и месяца нет - в середине года цена немного проседала, однако диапазон колебаний медианных значений невелик.

In [ ]:
data_pivot_y = data.pivot_table(index=['year'], values='last_price_mln', aggfunc=['median'])
data_pivot_y

Медианная цена в 2014 году была подозрительно высокой относительно других лет. 2015 год выше остальных лет (кроме 2014).

In [ ]:
data_pivot_y.plot()

In [ ]:
data_pivot_y_1 = data.pivot_table(index=['year'], values='last_price_mln', aggfunc=['count'])
data_pivot_y_1

In [ ]:
data_pivot_y_1.plot()

Видим, что данные за 2014-2015 не показательны (особенно за 2014 год) - выборка объявлений очень маленькая.
Медианная цена в 2016-2019 году (наибольшее количество объявлений) существенно не отличалась, лишь в 2019 году стала немного больше (возможно, из-за инфляции).
Таким образом, сформировать однозначный вывод о существенной зависимости цены от года не представляется возможным.
Однако, указанное обусловлено относительно коротким временным промежутком (2014-2019 гг.).
Вероятно, на более длинной дистанции корреляция была бы более очевидной.

В целом, если говорить о наиболее существенных факторах, влияющих на цену, из отраженных в задании, это общая площадь и жилая площадь квартиры. Большее количество комнат, большая площадь кухни и нахождение не на первом/последнем этаже также увеличивает стоимость, но не так линейно.
Возможно предположить, что существенное влияние на стоимость также оказывает близость к центру города, год постройки дома (отсутствует в базе) и неколичественные параметры (престижность района и уровень его инфраструктурной развитости, класс и состояние жилого комплекса/дома, качество отделки/ремонта квартиры).

**4.4 Определяем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений**

In [ ]:
data['locality_name'].value_counts().head(10) # определим 10 населенных пунктов с наибольшим числом объявлений.

Санкт-Петербург ожидаемо лидирует с большим отрывом.

In [ ]:
top_10 = data.query('locality_name in ["Санкт-Петербург", "поселок Мурино", "поселок Шушары", "Всеволожск", "Пушкин", "Колпино", "поселок Парголово", "Гатчина", "деревня Кудрово", "Выборг"]')
# сделаем срез из основной базы данных

In [ ]:
avg_meter = top_10['meter_price'].mean().round() # найдем среднее значение и округлим
avg_meter

Cредняя цена одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений - 108 972 рубля.

In [ ]:
top_10_meter = top_10.pivot_table(index=['locality_name'], values='meter_price', aggfunc=['mean']) # строим сводную таблицу
top_10_meter.columns = ['avg_meter_price']
top_10_meter['avg_meter_price'] = top_10_meter['avg_meter_price'].round()
top_10_meter.sort_values(by='avg_meter_price', ascending=False)

С помощью сводной таблицы определяем среднюю стоимость квадратного метра по населенным пунктам, затем сортируем данные.
Населённый пункт с самой высокой стоимостью квадратного метра - Санкт-Петербург (114 808 руб.), самой низкой - Выборг (58 142 руб.).
Предположительно, данный факт обусловлен тем, что Выборг - самый отдаленный от Санкт-Петербурга населенный пункт в выборке.

**4.5 Анализируем изменения цены квадратного метра в Санкт-Петербурге для каждого километра по степени удалённости от центра**

In [ ]:
spb = data.loc[data['locality_name'] == 'Санкт-Петербург'] # отберем квартиры, расположенные в Санкт-Петербурге

In [ ]:
spb_km = spb.pivot_table(index=['centre_km'], values='meter_price', aggfunc=['mean']) # вычисляем среднюю цену метра на каждый километр удаленности от центра Санкт-Петербурга
spb_km.columns = ['avg_meter_price_centre'] # переименовываем столбец
spb_km['avg_meter_price_centre'] = spb_km['avg_meter_price_centre'].round() # округляем значения
spb_km # выводим на экран

In [ ]:
spb_km.plot() # строим график

График в целом отражает обратную зависимость расстояния от центра города и стоимости квадратного метра.
Вместе с тем, по отдельным "километрам" наблюдаются скачки стоимости, обусловленные присутствием на данном расстоянии объектов, превосходящих по иным характеристикам более близкие к центру объекты.
Так, например, в 27 км от центра цена составляет 132,1 тыс. руб. - больше, чем в 8 км от центра.
Стоимость квадратного метра по мере отдаления от центра меняется нелинейно - где-то шаг составляет 20-30 тыс. руб., в иных случаях - 3-5 тыс. руб.

In [ ]:
spb_km_1 = spb.pivot_table(index=['centre_km'], values='meter_price', aggfunc=['count'])
spb_km_1 # выведем распределение количества объектов по километрам, полученное с помощью сводной таблицы

Мы видим, что выборка не симметрична - объектов ближе 10 км и дальше 19 км от центра относительно немного, в связи с чем средние данные по ним могут быть искажены под влиянием иных параметров (например, 20 и 27 км).

### Общий вывод

В ходе выполнения проекта было проведено исследование объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Задача анализа — установить параметры для определения рыночной стоимости в целях последующего построения автоматизированной системы. 

*В процессе работы*:
- были выявлены и удалены/заполнены пропущенные значения;
- изменены типы данных в некоторых столбцах;
- устранены неявные дубликаты в названиях населенных пунктов, а также аномальные значения в отдельных столбцах;
- в таблицу добавлены дополнительные расчетные показатели.
В целом можно отметить, что предобработка данных **существенным образом выборку не изменила** (удалено менее 1% записей), при этом ряд оставшихся **пропусков** (столбцы с расстоянием до центра города, ближайшего парка, аэропорта, водоема) не исказил результаты анализа (причина данных пропусков носит технический характер, кроме того, возможно в электронном справочнике по ряду населенных пунктов просто отсутствует точка центра города, не все парки и водоемы отмечены). Также пропуски **остались** в столбце с высотой потолка (учитывая их существенную долю от общего количества (около 40% или более 9 тыс.), соответствующей замены не нашлось).

*По итогам анализа установлено следующее*:

1. **Портрет среднестатистической квартиры таков**:
- однушка или двушка на 1-5 этаже пяти- или девятиэтажки (квартир на первом и последнем этажах меньше, чем на промежуточных);
- общая площадь около 45 кв. м;
- жилая площадь около 20 кв. м;
- площадь кухни около 6-10 кв. м;
- цена - около 4,5 млн руб.;
- высота потолка - около 2,5 м;
- квартира расположена в Санкт-Петербурге относительно недалеко от парка (400-600 м), однако явно не в центре города (в 13-15 км) и в отдалении от аэропорта (15-35 км).
Объявления в основном публикуются в будние дни, скорее не в летние и не в праздничные месяцы.

2. Продажа квартиры занимает **в большинстве случаев** 1-3,5 месяца. **Быстрыми** продажами можно считать сделки в пределах месяца. **Необычно долгие продажи** - после 8-9 месяцев.


3. Если говорить о **наиболее существенных факторах**, влияющих на цену, из отраженных в задании, это:
- **общая площадь и жилая площадь квартиры**. Корреляция - 0,65 и 0,585, соответственно;
- **большее количество комнат** (корреляция с ценой - 0,37), **большая площадь кухни** (корреляция с ценой - 0,51) и **нахождение не на первом/последнем этаже** (в меньшей степени) также увеличивает стоимость, но не так линейно (возможно, для установления более четкой корреляции необходим анализ по более симметричной выборке).

Возможно предположить, что **существенное влияние** на стоимость также оказывает год постройки дома (отсутствует в базе) и неколичественные параметры (престижность района и уровень его инфраструктурной развитости, класс и состояние жилого комплекса/дома, качество отделки/ремонта квартиры), также отсутствующие в базе данных.

4. **Близость к центру Санкт-Петербурга** в целом **положительно коррелирует** со стоимостью квартиры - стоимость квадратного метра по мере отдаления от центра снижается нелинейно - где-то шаг на 1 км составляет 20-30 тыс. руб., в иных случаях - 3-5 тыс. руб. (вместе с тем выборка **не в полной мере симметрична** - квартир близко к центру и совсем далеко от центра относительно немного). Кроме того, предположительно **расстояние иных населенных пунктов до Санкт-Петербурга** также имеет значение - например, в Выборге (находится дальше всех из числа наиболее популярных населенных пунктов) в среднем **самые дешевые квартиры** (58,1 тыс. руб. за 1 кв. м), **самые дорогие** - в Санкт-Петербурге - 114,8 тыс. руб. за 1 кв. м (вместе с тем, по данному вопросу целесообразно провести дополнительное исследование - возможно, дело не только в отдаленности от Санкт-Петербурга, но и в иных сопутствующих факторах (старый жилой фонд, неразвитая инфраструктура и т.д.)).

5. **Целесообразно** добавить в базу сведения о годе постройки дома и включить их в анализ - на мой взгляд, этот фактор мог бы стать одним из ключевых в части влияния на цену квартиры.
Кроме того, существенное влияние на стоимость могут оказывать вышеотмеченные качественные показатели, **отсутствующие в базе** (как минимум могли бы объяснить ряд выявленных фактов/тенденций).